#### Get a list of dicts with data

In [213]:
# import os

# os.chdir("./")

# direc = os.getcwd() # Get current working directory
# ext = '.json' # Select your file delimiter

# # Select only files with the ext extension
# txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]

In [5]:
# collect all small files into one list
import json


big_list = [] # Create an empty list, 

# Iterate over your json files
for f in txt_files:
    # Open them and assign k and v in them to big_dict
    with open(os.path.join(direc,f), 'r') as file_object:
        for i, item in enumerate(json.load(file_object)):
            big_list.append(item)
            
            
#             for key, value in item.items():
#                 big_list = value

In [6]:
big_list[0]

{'ent': 'ТОЗ',
 'wikibase': 'Q4449016',
 'what': 'страница значений в проекте Викимедиа',
 'wikidata': {'это частный случай понятия (P31)': 'страница значений в проекте Викимедиа (Q4167410)'},
 'claims': {'P31': ['Q4167410']},
 'description': 'страница значений',
 'label': 'ТОЗ'}

In [22]:
errors = []

for item_d in big_list:
    if 'wikidata' not in list(item_d.keys()):
        errors.append(item_d)
        

In [23]:
# не совпало с именами в вики

len(errors)

3604

In [28]:
# #remove errors

[big_list.remove(x) for x in errors]

In [195]:
big_list[1]

{'ent': 'Салов, Михаил Владимирович',
 'wikibase': 'Q18277228',
 'what': 'человек',
 'wikidata': {'это частный случай понятия (P31)': 'человек (Q5)',
  'дата рождения (P569)': '+1918-12-21T00:00:00Z',
  'дата смерти (P570)': '+1941-11-21T00:00:00Z',
  'место рождения (P19)': 'Нижегородская область (Q2246)',
  'полученные награды (P166)': ['орден Ленина (Q185493)',
   'орден Красного Знамени (Q337463)',
   'Герой Советского Союза (Q178473)'],
  'ID на портале «Герои страны» (P2943)': '4722',
  'война/сражение (P607)': 'Великая Отечественная война (Q189266)',
  'личное имя (P735)': 'Михаил (Q15621289)',
  'место смерти (P20)': 'Республика Карелия (Q1914)',
  'пол или гендер (P21)': 'мужской пол (Q6581097)',
  'принадлежность (P945)': 'СССР (Q15180)',
  'военная классификация потерь (P1347)': 'погиб в бою (Q210392)'},
 'aliases': ['Михаил Владимирович Салов'],
 'claims': {'P31': ['Q5'],
  'P569': ['+1918-12-21T00:00:00Z'],
  'P570': ['+1941-11-21T00:00:00Z'],
  'P19': ['Q2246'],
  'P166':

In [27]:
len(big_list)

27160

In [29]:
# os.chdir("./")

In [ ]:
# "what": "конфликт"
# "дата начала (P580)": "+1806-00-00T00:00:00Z",
# "дата окончания (P582)": "+1807-00-00T00:00:00Z",

# "what": "военный округ"    
# "дата основания/создания/возникновения (P571)": "+1916-03-00T00:00:00Z",
# "дата прекращения существования (P576)"
    
# человек 
# "дата рождения (P569)": "+1923-11-19T00:00:00Z",
# "дата смерти (P570)": "+1992-09-20T00:00:00Z",
    
# # 'момент времени (P585)' - НЕ ТРОГАЛА
# # 'основатель (P112)' - "Десятинная церковь"
    

#### Make a clean file with wikidata

In [1]:
import json

big_list = json.load(open("ent_27t_wdata_clean.json", "r", encoding="utf-8"))

In [216]:
all_q = questions_12 + questions_11 + questions_10 +questions_9 +questions_8+questions_7 +questions_6 +questions_5+questions_4+questions_3+questions_2+questions_1


In [217]:
# # save results

# with open('data_and_questions.json', "w", encoding="utf-8") as f:
#         json.dump(all_q, f, ensure_ascii=False, indent=4)

In [2]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
def infl(word:str, num:str, case:str) -> str:
    """Part of normalize()"""
    p = morph.parse(word)[0]
    return p.inflect({case})


def pos(word:str) -> str:
    """Part of normalize()"""
    p = morph.parse(word)[0]
    return p.tag.POS


def gender(word:str) -> str:
    """Part of normalize()"""
    p = morph.parse(word)[0]
    return p.tag.gender


def case(word:str) -> str:
    """Part of normalize()"""
    p = morph.parse(word)[0]
    return p.tag.case


def number(word:str) -> str:
    """Part of normalize()"""
    p = morph.parse(word)[0]
    return p.tag.number

def lemmatize(word:str) -> str:
    '''Part of normalize()

    Lemmatizes a word (for pymorphy2, context is irrelevant).
    Input: a string = a word
    Output: a string'''
    try:
        return "".join(morph.parse(word)[0].inflect({'sing', 'nomn'}).word).strip()
    except:
        return word.lower()

## different relation types / relations

In [4]:
# collect relation-what pairs

def make_rel_what(big_list, rel):
    category_of = []
    for di in big_list:
        for key, value in di['wikidata'].items():
    #         print(key)
            if 'what' not in list(di.keys()) or di['what'] == None:
                pass
            else:
                if key == rel:
                    category_of.append((key,di['what']))
    #             print(di['what'])
    return category_of


# sort tuples by frequency

def sort_fr(category_of):
        
    rel_freq = {}
    rel_what = {}
    for rel_what in category_of:
        if rel_what not in rel_freq:
            rel_freq[rel_what] = 0 
        rel_freq[rel_what] += 1
    return {k: v for k, v in sorted(rel_freq.items(), key=lambda item: item[1], reverse=True)}

In [9]:
# 'часть от (P361)' + what

role =  'часть от (P361)'

# sort_fr(make_rel_what(big_list, role))

In [60]:
# original
# 'часть от (P361)', 'осада')

import re

rel = 'часть от (P361)'
what = 'осада'

questions_15 = []

def make_q_p(big_list, rel, what):
    
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):
            pass
        else:
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
    #             print(di)
                try:
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
                    answer_open = di['wikidata'][rel].split('(')[0].strip()
                    atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
                    
                    d = {'type':'part_of', \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': atr, \
                        'open_q':['Во время какой войны', 'происходила', di['ent'], '?'], \
                        'answer_open':di['wikidata'][rel], \
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'происходила', 'во время',  answer_open, '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_15 = make_q_p(big_list, rel, what)
# questions_all += questions_2
len(questions_15)

57

In [ ]:
# beautify open questions
def boc(list_of_dicts):
    for d in list_of_dicts:
        words = d['open_q'][-2].split()
        if number(words[0]) == 'plur':
            d['open_q'][-3] = "произошли"
        elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
            d['open_q'][-3] = "произошла"
        elif any([f for f in words if f.endswith('е')]):
            d['open_q'][-3] = "произошло"
        else:
            d['open_q'][-3] = "произошёл"
        print(d['open_q'])

# beautify closed questions
def bcc(list_of_dicts):
    for d in list_of_dicts:
        words = d['closed_q'][-2].split()
        cased_words = []
        for i, word in enumerate(words):
            if pos(word) == 'NOUN' and number(word) == 'sing':
                cased_words.append(infl(word, 'sing', 'gent')[0])
               
                if i > 0 and len(cased_words) > 1 and gender(word) == 'femn' and cased_words[-2].endswith('го') :
    #                 print('go case!! tur')
                    last = cased_words.pop()
    #                 print('go', last)
                    adj = cased_words.pop()
                    cased_words.append(adj[:-2] +'й')
                    cased_words.append(last)
    
                elif len(cased_words) == 0 and len(words) > 1 and i > 1:
                    if number(word) == 'plur' and words[i-1].endswith('ие'):
    
                        cased_words.append(words[i-1][:-2] +'их')
                    elif gender(word) == 'masc' and words[i-1].endswith('ий'):
    
                        cased_words.append(words[i-1][:-3] +'ого')
                    elif gender(word) == 'femn' and words[i-1].endswith('ая'):
    
                        cased_words.append(words[i-1][:-2] +'ой')
                    elif gender(word) == 'neut' and words[i-1].endswith('ое'):
    
                        cased_words.append(words[i-1][:-2] +'ого')    
                break
            elif pos(word) == 'NOUN' and number(word) == 'plur':
                cased_words.append(infl(word, 'plur', 'gent')[0])
                break
            elif pos(word) == 'ADJF' and number(word) == 'sing' :
                cased_words.append(infl(word, 'sing', 'gent')[0][0].upper() + infl(word, 'sing', 'gent')[0][1:])
            elif pos(word) == 'ADJF' and number(word) == 'sing':
                cased_words.append(infl(word, 'plur', 'gent')[0][0].upper() + infl(word, 'plur', 'gent')[0][1:])
            else:
                cased_words.append(word)
                
        if len(cased_words) < len(words):
    #         print(cased_words, words)
            for word in words[len(cased_words):]:
                cased_words.append(word)
        
        d['closed_q'][-2] = ' '.join(cased_words)
        d ['closed_q'][1] = d['open_q'][-3]
    #     print(d['closed_q'], d['open_q'])
            

In [533]:
# beautify open questions

for d in questions_15:
    words = d['open_q'][-2].split()
    if number(words[0]) == 'plur':
        d['open_q'][-3] = "произошли"
    elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
        d['open_q'][-3] = "произошла"
    elif any([f for f in words if f.endswith('е')]):
        d['open_q'][-3] = "произошло"
    else:
        d['open_q'][-3] = "произошёл"
        

# ' '.join(

In [62]:
what = 'человек'
rel = 'испытал влияние от (P737)'
make_q_p(big_list, rel, what)

[{'type': 'part_of',
  'what': 'человек',
  'ent': 'Чашник, Илья Григорьевич',
  'Q': 'Q1658589',
  'rel_r': 'испытал влияние от (P737)',
  'rel': 'P737',
  'atr': 'Q130777',
  'open_q': ['Во время какой войны',
   'происходила',
   'Чашник, Илья Григорьевич',
   '?'],
  'answer_open': 'Казимир Малевич',
  'closed_q': ['Чашник, Илья Григорьевич',
   'происходила',
   'во время',
   'Казимир Малевич',
   '.'],
  'answer_closed': 'Да'},
 {'type': 'part_of',
  'what': 'человек',
  'ent': 'Маковский, Константин Егорович',
  'Q': 'Q460337',
  'rel_r': 'испытал влияние от (P737)',
  'rel': 'P737',
  'atr': 'Q4768',
  'open_q': ['Во время какой войны',
   'происходила',
   'Маковский, Константин Егорович',
   '?'],
  'answer_open': 'Карл Павлович Брюллов',
  'closed_q': ['Маковский, Константин Егорович',
   'происходила',
   'во время',
   'Карл Павлович Брюллов',
   '.'],
  'answer_closed': 'Да'},
 {'type': 'part_of',
  'what': 'человек',
  'ent': 'Коровин, Константин Алексеевич',
  'Q': 'Q4

In [537]:
# beautify closed questions

for d in questions_15:
    words = d['closed_q'][-2].split()
    cased_words = []
    for i, word in enumerate(words):
        if pos(word) == 'NOUN' and number(word) == 'sing':
            cased_words.append(infl(word, 'sing', 'gent')[0])
            
            if i > 0 and len(cased_words) > 1 and gender(word) == 'femn' and cased_words[-2].endswith('го') :
                print('go case!! tur')
                last = cased_words.pop()
                print('go', last)
                adj = cased_words.pop()
                cased_words.append(adj[:-2] +'й')
                cased_words.append(last)
                print
            elif len(cased_words) == 0 and len(words) > 1 and i > 1:
                if number(word) == 'plur' and words[i-1].endswith('ие'):

                    cased_words.append(words[i-1][:-2] +'их')
                elif gender(word) == 'masc' and words[i-1].endswith('ий'):

                    cased_words.append(words[i-1][:-3] +'ого')
                elif gender(word) == 'femn' and words[i-1].endswith('ая'):

                    cased_words.append(words[i-1][:-2] +'ой')
                elif gender(word) == 'neut' and words[i-1].endswith('ое'):

                    cased_words.append(words[i-1][:-2] +'ого')    
            break
        elif pos(word) == 'NOUN' and number(word) == 'plur':
            cased_words.append(infl(word, 'plur', 'gent')[0])
            break
        elif pos(word) == 'ADJF' and number(word) == 'sing' :
            cased_words.append(infl(word, 'sing', 'gent')[0][0].upper() + infl(word, 'sing', 'gent')[0][1:])
        elif pos(word) == 'ADJF' and number(word) == 'sing':
            cased_words.append(infl(word, 'plur', 'gent')[0][0].upper() + infl(word, 'plur', 'gent')[0][1:])
        else:
            cased_words.append(word)
            
    if len(cased_words) < len(words):
        print(cased_words, words)
        for word in words[len(cased_words):]:
            cased_words.append(word)
    
    d['closed_q'][-2] = ' '.join(cased_words)
    d ['closed_q'][1] = d['open_q'][-3]
    print(d['closed_q'], d['open_q'])
        

['Осада Выборга (1710)', 'произошла', 'во время', 'Северной войны', '.'] ['Во время какой войны', 'произошла', 'Осада Выборга (1710)', '?']
['Монгольского', 'нашествия'] ['монгольское', 'нашествие', 'на', 'Русь']
['Оборона Владимира', 'произошла', 'во время', 'Монгольского нашествия на Русь', '.'] ['Во время какой войны', 'произошла', 'Оборона Владимира', '?']
go case!! tur
go войны
['Осада Свеаборга', 'произошла', 'во время', 'Русско-шведской войны', '.'] ['Во время какой войны', 'произошла', 'Осада Свеаборга', '?']
['Осада Таганрога', 'произошла', 'во время', 'Крымской войны', '.'] ['Во время какой войны', 'произошла', 'Осада Таганрога', '?']
['Блокада Ленинграда', 'произошла', 'во время', 'Великой Отечественной войны', '.'] ['Во время какой войны', 'произошла', 'Блокада Ленинграда', '?']
['Монгольского', 'нашествия'] ['монгольское', 'нашествие', 'на', 'Русь']
['Битва под Коломной (1238)', 'произошла', 'во время', 'Монгольского нашествия на Русь', '.'] ['Во время какой войны', 'произ

In [538]:
# save results

# os.chdir("../")
# os.chdir("questions_open")

# with open('part_.json', "w", encoding="utf-8") as f:
#         json.dump(questions_15, f, ensure_ascii=False, indent=4)

In [289]:
!ls

part_osada.json


# Конфликт

In [539]:
rel = 'часть от (P361)'
what = 'конфликт'
pk = make_q_p(big_list, rel, what)

['Во время какой войны', 'происходила', 'Инцидент на мосту Лугоу', '?']
['Во время какой войны', 'происходила', 'Ливонская война', '?']
['Во время какой войны', 'происходила', 'Война четвёртой коалиции', '?']
['Во время какой войны', 'происходила', 'Смоленская война', '?']
['Во время какой войны', 'происходила', 'Подавление Венгерского восстания (1848—1849)', '?']
['Во время какой войны', 'происходила', 'Битва при Переде', '?']
['Во время какой войны', 'происходила', 'Инцидент с сухогрузом «Пауэрфул»', '?']
['Во время какой войны', 'происходила', 'Конфликт на Китайско-Восточной железной дороге', '?']
['Во время какой войны', 'происходила', 'Русско-турецкая война (1806—1812)', '?']
['Во время какой войны', 'происходила', 'Ичкеринское сражение', '?']
['Во время какой войны', 'происходила', 'Южно-Сахалинская операция (1945)', '?']
['Во время какой войны', 'происходила', 'Конфликт в Южном Ливане (1985—2000)', '?']
['Во время какой войны', 'происходила', 'Интервенция в Синьцзян', '?']
['Во 

In [550]:
# beautify open questions

for d in pk:
#     print(d['open_q'][-2].split())
#     print(any([f for f in d['open_q'][-2].split() if f.endswith('а')]))
#     print([f for f in d['open_q'][-2].split() if f.endswith('а')])
    words = d['open_q'][-2].split()
    if number(words[0]) == 'plur':
        d['open_q'][-3] = "произошли"
    elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
        d['open_q'][-3] = "произошла"
    elif any([f for f in words if f.endswith('е')]):
        d['open_q'][-3] = "произошло"
    else:
        d['open_q'][-3] = "произошёл"
#     print(d['open_q'])

# ' '.join(

In [551]:
# beautify closed questions

for d in pk:
    words = d['closed_q'][-2].split()
    cased_words = []
    for i, word in enumerate(words):
        if pos(word) == 'NOUN' and number(word) == 'sing':
            cased_words.append(infl(word, 'sing', 'gent')[0])
           
            if i > 0 and len(cased_words) > 1 and gender(word) == 'femn' and cased_words[-2].endswith('го') :
                print('go case!! tur')
                last = cased_words.pop()
                print('go', last)
                adj = cased_words.pop()
                cased_words.append(adj[:-2] +'й')
                cased_words.append(last)

            elif len(cased_words) == 0 and len(words) > 1 and i > 1:
                if number(word) == 'plur' and words[i-1].endswith('ие'):

                    cased_words.append(words[i-1][:-2] +'их')
                elif gender(word) == 'masc' and words[i-1].endswith('ий'):

                    cased_words.append(words[i-1][:-3] +'ого')
                elif gender(word) == 'femn' and words[i-1].endswith('ая'):

                    cased_words.append(words[i-1][:-2] +'ой')
                elif gender(word) == 'neut' and words[i-1].endswith('ое'):

                    cased_words.append(words[i-1][:-2] +'ого')    
            break
        elif pos(word) == 'NOUN' and number(word) == 'plur':
            cased_words.append(infl(word, 'plur', 'gent')[0])
            break
        elif pos(word) == 'ADJF' and number(word) == 'sing' :
            cased_words.append(infl(word, 'sing', 'gent')[0][0].upper() + infl(word, 'sing', 'gent')[0][1:])
        elif pos(word) == 'ADJF' and number(word) == 'sing':
            cased_words.append(infl(word, 'plur', 'gent')[0][0].upper() + infl(word, 'plur', 'gent')[0][1:])
        else:
            cased_words.append(word)
            
    if len(cased_words) < len(words):
#         print(cased_words, words)
        for word in words[len(cased_words):]:
            cased_words.append(word)
    
    d['closed_q'][-2] = ' '.join(cased_words)
    d ['closed_q'][1] = d['open_q'][-3]
#     print(d['closed_q'], d['open_q'])
        

In [542]:

# with open('part_konfl.json', "w", encoding="utf-8") as f:
#         json.dump(pk, f, ensure_ascii=False, indent=4)

# Битва

In [610]:
rel = 'часть от (P361)'
what = 'битва'
pb = make_q_p(big_list, rel, what)

In [611]:
boc(pb)

['Во время какой войны', 'произошло', 'Тверское сражение', '?']
['Во время какой войны', 'произошло', 'Белостокско-Минское сражение', '?']
['Во время какой войны', 'произошёл', 'Бредовский поход', '?']
['Во время какой войны', 'произошло', 'Сражение при Драмдаге', '?']
['Во время какой войны', 'произошла', 'Восточно-Прусская операция (1914)', '?']
['Во время какой войны', 'произошла', 'Варшавская битва (1920)', '?']
['Во время какой войны', 'произошла', 'Битва под Ленино', '?']
['Во время какой войны', 'произошла', 'Битва под Студзянками', '?']
['Во время какой войны', 'произошёл', 'Бой под Боремелем', '?']
['Во время какой войны', 'произошла', 'Варшавская заутреня', '?']
['Во время какой войны', 'произошла', 'Битва при Нанси', '?']
['Во время какой войны', 'произошла', 'Невская битва', '?']
['Во время какой войны', 'произошёл', 'Бой под Цёлково', '?']
['Во время какой войны', 'произошло', 'Сражение на Чёрной речке', '?']
['Во время какой войны', 'произошла', 'Сражение при Козлуджи', '

['Во время какой войны', 'произошла', 'Урмийская операция', '?']
['Во время какой войны', 'произошла', 'Битва при Хагельберге', '?']
['Во время какой войны', 'произошла', 'Хамаданская операция', '?']
['Во время какой войны', 'произошла', 'Ханекино-Хамаданская операция', '?']
['Во время какой войны', 'произошла', 'Оборона Харбина (1900)', '?']
['Во время какой войны', 'произошло', 'Сражение при Хёрнефорсе', '?']
['Во время какой войны', 'произошла', 'Вторая битва при Цюрихе', '?']
['Во время какой войны', 'произошло', 'Сражение при Шампобере', '?']
['Во время какой войны', 'произошло', 'Сражение при Шато-Тьерри (1814)', '?']
['Во время какой войны', 'произошёл', 'Штурм Гродно (1708)', '?']
['Во время какой войны', 'произошла', 'Эрзинджанская операция', '?']
['Во время какой войны', 'произошло', 'Взятие Эривани (1827)', '?']
['Во время какой войны', 'произошло', 'Сражение у Якобштадта', '?']
['Во время какой войны', 'произошли', 'Бои за Аргун (1999—2000)', '?']
['Во время какой войны', '

['Во время какой войны', 'произошло', 'Освобождение Ченстоховы', '?']
['Во время какой войны', 'произошла', 'Спас-Деменская операция', '?']
['Во время какой войны', 'произошло', 'Сражение под Сливице', '?']
['Во время какой войны', 'произошло', 'Сражение при Соколово', '?']
['Во время какой войны', 'произошла', 'Стычка при Курилёвке', '?']
['Во время какой войны', 'произошёл', 'Бой у Техумарди', '?']
['Во время какой войны', 'произошла', 'Битва при Рущуке', '?']
['Во время какой войны', 'произошёл', 'Бой под Боремелем', '?']
['Во время какой войны', 'произошёл', 'Бой у Будзиски', '?']
['Во время какой войны', 'произошло', 'Первое сражение под Вавром', '?']
['Во время какой войны', 'произошла', 'Осада Варшавы (1831)', '?']
['Во время какой войны', 'произошла', 'Полянская битва', '?']
['Во время какой войны', 'произошла', 'Сараевская операция', '?']
['Во время какой войны', 'произошла', 'Битва на Сутьеске', '?']
['Во время какой войны', 'произошло', 'Сражение при Келье', '?']
['Во время 

In [612]:
bcc(pb)

In [613]:
len(pb)

555

In [615]:
# pb[:10]

In [616]:

with open('part_bitva.json', "w", encoding="utf-8") as f:
        json.dump(pb, f, ensure_ascii=False, indent=4)

# Война

In [598]:
rel = 'часть от (P361)'
what = 'война'
pv = make_q_p(big_list, rel, what)

In [600]:
# boc(pv)

In [601]:
bcc(pv)

In [604]:
# pv

In [603]:

with open('part_voyna.json', "w", encoding="utf-8") as f:
        json.dump(pv, f, ensure_ascii=False, indent=4)

#### date

In [81]:
date_types = []

for di in big_list:
    for key, value in di['wikidata'].items():
        if 'дата ' in key: 
            date_types.append(key) #,di['ent']
#             print(di['what'])

In [82]:
#unique date types
date_types_u = list(set(date_types))
date_types_u

['дата принятия в эксплуатацию (P729)',
 'дата официального открытия (P1619)',
 'дата прекращения существования (P576)',
 'дата крещения (P1636)',
 'дата коммерциализации (P5204)',
 'дата посадки (P620)',
 'дата смерти (P570)',
 'дата рождения (P569)',
 'дата исчезновения (P746)',
 'дата окончания (P582)',
 'дата официального закрытия (P3999)',
 'дата открытия (P575)',
 'дата основания/создания/возникновения (P571)',
 'дата начала (P580)',
 'дата запуска космического корабля (P619)',
 'дата схода с орбиты (P621)',
 'дата погребения или кремации (P4602)',
 'минимальный возраст кандидата (P2998)',
 'дата вывода из эксплуатации (P730)',
 'дата публикации (P577)']

In [71]:
dates = []

for di in big_list:
    for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):
            pass
        else:
            if 'дата ' in key: 
                dates.append((key,di['what'])) #,di['ent']
#             print(di['what'])

In [72]:
# what = P31 = instance of

In [104]:
# all  entities with dates with what = P31 = instance of
len(dates)

34895

In [3]:
datefreq = {}
datewhat = {}
for rel_what in dates:
    if rel_what not in datefreq:
        datefreq[rel_what] = 0 
    datefreq[rel_what] += 1
   
    
# sort by frequency

{k: v for k, v in sorted(datefreq.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
# Out:
# ('дата рождения (P569)', 'человек'): 15207,
#  ('дата смерти (P570)', 'человек'): 14033,
#  ('дата публикации (P577)', 'фильм'): 221,
#  ('дата начала (P580)', 'битва'): 201,
#  ('дата окончания (P582)', 'битва'): 199,

In [ ]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_hram_start18-.json', "w", encoding="utf-8") as f:
    json.dump(questions_18, f, ensure_ascii=False, indent=4)

In [206]:
# даты

len(questions_12+ questions_11 + questions_10 +questions_9 +questions_8+questions_7 +questions_6 +questions_5+questions_4+questions_3+questions_2+questions_1)

929

In [203]:
# ('дата публикации (P577)', 'письменное произведение'): 43,

import re
# надо ли? 

rel = 'дата публикации (P577)'
what = 'письменное произведение'

questions_11 = []

def make_q(big_list, rel, what):
    
    questions = []
    
    for di in big_list:
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):
            pass
        else:
            if di['what'] == what and rel in di['wikidata']:
    #             print(di)
                try:
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
                    
                    if type(di['wikidata']['жанр (P136)']) ==list:
                        zhanr = di['wikidata']['жанр (P136)'][0]
                    else:
                        zhanr = di['wikidata']['жанр (P136)']
                    
                    d = {'type':'date', \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'open_q':['В каком году', 'был создан', zhanr, '"',di['ent'], '"', '?'], \
                        'answer_open':di['wikidata'][rel], \
                        'closed_q':[zhanr, di['ent'], 'был создан','в', di['wikidata'][rel], 'году.'], \
                        'answer_open':'Да'
                                       }
                    questions.append(d)
#                     print(d)
                    print(d['open_q'], d['closed_q'])
                    print(' ')
                except:
                    pass
    return questions

questions_11 = make_q(big_list, rel, what)
# questions_all += questions_2
len(questions_11)

['В каком году', 'был создан', 'автобиографический роман (Q3056541)', '"', 'В круге первом', '"', '?'] ['автобиографический роман (Q3056541)', 'В круге первом', 'был создан', 'в', '+1968-00-00T00:00:00Z', 'году.']
 
['В каком году', 'был создан', 'детская литература (Q131539)', '"', 'Удивительное путешествие Нильса Хольгерссона с дикими гусями по Швеции', '"', '?'] ['детская литература (Q131539)', 'Удивительное путешествие Нильса Хольгерссона с дикими гусями по Швеции', 'был создан', 'в', ['+1906-00-00T00:00:00Z', '+1907-00-00T00:00:00Z'], 'году.']
 
['В каком году', 'был создан', 'история России (Q161414)', '"', 'История государства Российского', '"', '?'] ['история России (Q161414)', 'История государства Российского', 'был создан', 'в', '+1818-00-00T00:00:00Z', 'году.']
 
['В каком году', 'был создан', 'роман (Q8261)', '"', 'Овод (роман)', '"', '?'] ['роман (Q8261)', 'Овод (роман)', 'был создан', 'в', '+1897-00-00T00:00:00Z', 'году.']
 
['В каком году', 'был создан', 'роман (Q8261)',

38

In [52]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'храм'

import re


questions_18 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был основан', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'был основан', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_18 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_18)


42

In [56]:
# best beautify open questions

for d in questions_18:
    if d['open_q'][-2].split(',')[0].endswith('а') \
    or d['open_q'][-2].split(',')[0].endswith('я') \
    or d['open_q'][-2].split()[0].endswith('а') \
    or d['open_q'][-2].split()[0].endswith('ь') \
    or d['open_q'][-2].split()[0].endswith('я'):

        d['open_q'][1] = d['closed_q'][1] = 'была основана'
    elif d['open_q'][-2].split()[0].endswith('е') \
        or d['open_q'][-2].split()[0].endswith('о'):
        d['open_q'][1] = d['closed_q'][1] = 'было основано'
        
    elif d['open_q'][-2].split()[0].endswith('и') \
        or d['open_q'][-2].split()[0].endswith('ы'):
        d['open_q'][1] = d['closed_q'][1] = 'были основаны'
    
    else:
        d['open_q'][1] = d['closed_q'][1] = 'был основан'
        
for di in questions_18:
    print(di['open_q'], di['closed_q'])

['В каком году', 'была основана', 'Церковь Святой Екатерины (Феодосия)', '?'] ['Церковь Святой Екатерины (Феодосия)', 'была основана', 'в', '1892', 'году', '.']
['В каком году', 'была основана', 'Церковь Покрова на Нерли', '?'] ['Церковь Покрова на Нерли', 'была основана', 'в', '1166', 'году', '.']
['В каком году', 'была основана', 'Церковь Петра и Павла на Городянке', '?'] ['Церковь Петра и Павла на Городянке', 'была основана', 'в', '1146', 'году', '.']
['В каком году', 'была основана', 'Церковь Святой Богородицы (Самарканд)', '?'] ['Церковь Святой Богородицы (Самарканд)', 'была основана', 'в', '1903', 'году', '.']
['В каком году', 'была основана', 'Церковь Покрова на Нерли', '?'] ['Церковь Покрова на Нерли', 'была основана', 'в', '1166', 'году', '.']
['В каком году', 'был основан', 'Храм Трёх Святителей на Кулишках', '?'] ['Храм Трёх Святителей на Кулишках', 'был основан', 'в', '1670', 'году', '.']
['В каком году', 'был основан', 'Собор Иоанна Предтечи (Псков)', '?'] ['Собор Иоанна П

In [57]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_hram_start18-.json', "w", encoding="utf-8") as f:
    json.dump(questions_18, f, ensure_ascii=False, indent=4)

In [45]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'организация'

import re


questions_17 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был основан', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'был основан', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_17 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_17)


25

In [50]:
# best beautify open questions

for d in questions_17:
    if d['open_q'][-2].split(',')[0].endswith('а') \
    or d['open_q'][-2].split(',')[0].endswith('я') \
    or d['open_q'][-2].split()[0].endswith('а') \
    or d['open_q'][-2].split()[0].endswith('я'):

        d['open_q'][1] = d['closed_q'][1] = 'была основана'
    elif d['open_q'][-2].split()[0].endswith('е') \
        or d['open_q'][-2].split()[0].endswith('о'):
        d['open_q'][1] = d['closed_q'][1] = 'было основано'
    
    else:
        d['open_q'][1] = d['closed_q'][1] = 'был основан'
        
# for di in questions_17:
#     print(di['open_q'], di['closed_q'])

In [49]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_org_start17.json', "w", encoding="utf-8") as f:
    json.dump(questions_17, f, ensure_ascii=False, indent=4)

In [167]:
# 'дата начала (P580)', 'конфликт'
# не нравится( 

import re

rel = 'дата начала (P580)'
what = 'конфликт'

questions_8 = []

def make_q(big_list, rel, what):
    
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):
            pass
        else:
            if di['what'] == what and rel in di['wikidata']:
    #             print(di)
                try:
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
                    
                    d = {'type':'date', \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'open_q':['В каком году', 'начался', what, di['ent'], '?'], \
                        'answer_open':di['wikidata'][rel], \
                        'closed_q':[what, di['ent'], 'начался', 'в',  di['wikidata'][rel], 'году.'], \
                        'answer_open':'Да'
                                       }
                    questions.append(d)
#                     print(d)
                    print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 200:
            break
    return questions

questions_8 = make_q(big_list, rel, what)
# questions_all += questions_2
len(questions_8)

['В каком году', 'начался', 'конфликт', 'Дворцовые перевороты в Османской империи (1807—1808)', '?'] ['конфликт', 'Дворцовые перевороты в Османской империи (1807—1808)', 'начался', 'в', '+1807-00-00T00:00:00Z', 'году.']
['В каком году', 'начался', 'конфликт', 'Инцидент на мосту Лугоу', '?'] ['конфликт', 'Инцидент на мосту Лугоу', 'начался', 'в', '+1937-07-07T00:00:00Z', 'году.']
['В каком году', 'начался', 'конфликт', 'Ливонская война', '?'] ['конфликт', 'Ливонская война', 'начался', 'в', '+1558-01-17T00:00:00Z', 'году.']
['В каком году', 'начался', 'конфликт', 'Война четвёртой коалиции', '?'] ['конфликт', 'Война четвёртой коалиции', 'начался', 'в', '+1806-00-00T00:00:00Z', 'году.']
['В каком году', 'начался', 'конфликт', 'Битва при Переде', '?'] ['конфликт', 'Битва при Переде', 'начался', 'в', '+1849-06-20T00:00:00Z', 'году.']
['В каком году', 'начался', 'конфликт', 'Конфликт на Китайско-Восточной железной дороге', '?'] ['конфликт', 'Конфликт на Китайско-Восточной железной дороге', 'н

55

In [40]:
rel = 'дата прекращения существования (P576)'
what = 'историческое государство'

import re


questions_16 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'закончил своё существование', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'закончил своё существование', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_16 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_16)


80

In [39]:
# beautify open questions

for d in questions_16:
    if d['open_q'][-2].split(',')[0].endswith('а') \
    or d['open_q'][-2].split(',')[0].endswith('я') \
    or d['open_q'][-2].split()[0].endswith('а') \
    or d['open_q'][-2].split()[0].endswith('я'):

        d['open_q'][1] = d['closed_q'][1] = 'закончила своё существование'
    elif d['open_q'][-2].split()[0].endswith('е'):
        d['open_q'][1] = d['closed_q'][1] = 'закончило своё существование'
    
    else:
        d['open_q'][1] = d['closed_q'][1] = 'закончил своё существование'
        
# for di in questions_16:
#     print(di['open_q'], di['closed_q'])

SyntaxError: unexpected EOF while parsing (<ipython-input-39-999892238e80>, line 16)

In [38]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_state_end16-.json', "w", encoding="utf-8") as f:
    json.dump(questions_16, f, ensure_ascii=False, indent=4)

In [25]:
rel = 'дата рождения (P569)'
# rel = 'участвовал в (P1344)'
what = 'человек'


import re


questions_15 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'родился', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'родился', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_15 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_15)


1000

In [19]:
# beautify open questions

for d in questions_15:
    if d['open_q'][-2].split(',')[0].endswith('а') \
    or d['open_q'][-2].split(',')[0].endswith('я') \
    or d['open_q'][-2].split()[0].endswith('а'):

        d['open_q'][1] = d['closed_q'][1] = "родилась"
    else:
        d['open_q'][1] = d['closed_q'][1] = "родился"

In [22]:

# for di in questions_15:
#     print(di['open_q'], di['closed_q'])

In [21]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_birth15--.json', "w", encoding="utf-8") as f:
    json.dump(questions_15, f, ensure_ascii=False, indent=4)

In [110]:
rel = 'дата публикации (P577)'
what = 'литературное произведение'

import re


questions_14 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
                    if type(di['wikidata']['жанр (P136)']) ==list:
                        zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
                    else:
                        zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был опубликован', zhanr, '"', di['ent'], '"','?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[zhanr, '"',di['ent'],'"', 'был опубликован', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
                    print(d['open_q'], d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_14 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_14)


['В каком году', 'был опубликован', 'роман в стихах', '"', 'Евгений Онегин', '"', '?'] ['роман в стихах', '"', 'Евгений Онегин', '"', 'был опубликован', 'в', '1825', 'году', '.']
['В каком году', 'был опубликован', 'Наррация', '"', 'Алексиада', '"', '?'] ['Наррация', '"', 'Алексиада', '"', 'был опубликован', 'в', '1148', 'году', '.']
['В каком году', 'был опубликован', 'автобиография', '"', 'Берлинское детство на рубеже веков', '"', '?'] ['автобиография', '"', 'Берлинское детство на рубеже веков', '"', 'был опубликован', 'в', '1950', 'году', '.']
['В каком году', 'был опубликован', 'книга путешествий', '"', 'Путешествие из Петербурга в Москву', '"', '?'] ['книга путешествий', '"', 'Путешествие из Петербурга в Москву', '"', 'был опубликован', 'в', '1790', 'году', '.']
['В каком году', 'был опубликован', 'роман', '"', 'Как закалялась сталь', '"', '?'] ['роман', '"', 'Как закалялась сталь', '"', 'был опубликован', 'в', '1932', 'году', '.']
['В каком году', 'был опубликован', 'роман', '"',

40

In [111]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_published14.json', "w", encoding="utf-8") as f:
    json.dump(questions_14, f, ensure_ascii=False, indent=4)

In [107]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'литературное произведение'

import re


questions_13 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
                    if type(di['wikidata']['жанр (P136)']) ==list:
                        zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
                    else:
                        zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был создан', zhanr, '"', di['ent'], '"','?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[zhanr, '"',di['ent'],'"', 'был создан', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_13 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_13)


27

In [ ]:
# beautify open questions
def boc2(list_of_dicts):
    for d in list_of_dicts:
        # make a list of the entity string
        words = d['open_q'][-2].split()
        if number(words[0]) == 'plur':
            d['open_q'][-3] = d['closed_q'][-5] = "были основаны"
        elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
            d['open_q'][-3] = d['closed_q'][-5] = "была основана"
        elif any([f for f in words if f.endswith('е')]):
            d['open_q'][-3] = d['closed_q'][-5] = "было основано"
        else:
            d['open_q'][-3] = d['closed_q'][-5] = "был основан"
        print(d['open_q'])

In [101]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_literat13--.json', "w", encoding="utf-8") as f:
    json.dump(questions_13, f, ensure_ascii=False, indent=4)

In [95]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'памятник'

import re


questions_12 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
                    
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был создан', '"', di['ent'], '"','?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':['"',di['ent'],'"', 'был создан', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_12 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_12)


26

In [94]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_pamyatnik12-.json', "w", encoding="utf-8") as f:
    json.dump(questions_12, f, ensure_ascii=False, indent=4)

In [90]:
# original

import re

rel = 'создатель (P170)'
what = 'картина'

questions_11 = []

def make_q_p(big_list, rel, what):
    
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) or '(' in di['ent']:
            pass
        else:
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
    #             print(di)
                try:
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
                    answer_open = di['wikidata'][rel].split('(')[0].strip()
                    atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
                    
                    d = {'type':'author', \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': atr, \
                        'open_q':['Кто написал картину', '"', di['ent'],'"', '?'], \
                        'answer_open':di['wikidata'][rel], \
                        'answer_open': answer_open, \
                        'closed_q':['Картину', '"', di['ent'],'"', 'написал',  answer_open, '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_11 = make_q_p(big_list, rel, what)
len(questions_11)


34

In [91]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('auth_kartina11.json', "w", encoding="utf-8") as f:
    json.dump(questions_11, f, ensure_ascii=False, indent=4)

In [86]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'картина'

import re


questions_10 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
                    
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
                    
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': ['создатель (P170)', di['wikidata']['создатель (P170)']], \
                        'open_q':['В каком году', 'была написана',what, '"', di['ent'], '"','?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[what,'"',di['ent'],'"', 'была написана', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_10 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_10)


53

In [74]:
len(questions_10)

53

In [75]:
# save results

import os

# os.chdir("../")

# os.chdir("questions_open")

with open('date_kartina---.json', "w", encoding="utf-8") as f:
    json.dump(questions_10, f, ensure_ascii=False, indent=4)

In [54]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'город'

import re


questions_9 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) : #or '(' in di['ent']
            pass
        else:
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
    #             print(di)
                try:
                    
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
                    
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'была основана', di['ent'], '?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'была основана', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_9 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_9)

54

In [55]:
len(questions_9)

54

In [56]:
# beautify open questions
def boc1(list_of_dicts):
    for d in list_of_dicts:
        # make a list of the entity string
        words = d['open_q'][-2].split()
        if number(words[0]) == 'plur':
            d['open_q'][-3] = d['closed_q'][-5] = "были основаны"
        elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
            d['open_q'][-3] = d['closed_q'][-5] = "была основана"
        elif any([f for f in words if f.endswith('е')]):
            d['open_q'][-3] = d['closed_q'][-5] = "было основано"
        else:
            d['open_q'][-3] = d['closed_q'][-5] = "был основан"
        print(d['open_q'])

In [58]:
# boc1(questions_9)

In [ ]:
'Усть-Каменогорск', 'Петах-Тиква', 

In [51]:
# # save results

# import os

# # os.chdir("../")

# # os.chdir("questions_open")

# with open('start_gorod9---.json', "w", encoding="utf-8") as f:
#     json.dump(questions_9, f, ensure_ascii=False, indent=4)

In [146]:
len(questions_1)

201

In [40]:
# 147 saved

import re

rel = 'дата начала (P580)'
what = 'битва'

questions_15 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) or '(' in di['ent']:
            pass
        else:
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
    #             print(di)
                try:
                    
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
                    
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'началась', di['ent'], '?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'началась', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(d)
#                     print(d['open_q'])#, d['closed_q'])
                except:
                    pass
#         if len(questions) == 200:
#             break
    return questions

questions_15 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_15)

148

In [31]:
len(questions_15)

148

In [33]:
# beautify open questions
def boc(list_of_dicts):
    for d in list_of_dicts:
        # make a list of the entity string
        words = d['open_q'][-2].split()
        if number(words[0]) == 'plur':
            d['open_q'][-3] = "начались"
            d['closed_q'][-5] = "начались"
        elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
            d['open_q'][-3] = "началась"
            d['closed_q'][-5] = "началась"
        elif any([f for f in words if f.endswith('е')]):
            d['open_q'][-3] = "началось"
            d['closed_q'][-5] = "началось"
        else:
            d['open_q'][-3] = "начался"
            d['closed_q'][-5] = "начался"
        print(d['open_q'])

In [39]:
# questions_15

In [34]:
boc(questions_15)

['В каком году', 'началось', 'Белостокско-Минское сражение', '?']
['В каком году', 'началась', 'Битва под Ленино', '?']
['В каком году', 'началась', 'Битва под Студзянками', '?']
['В каком году', 'началась', 'Битва при Газале', '?']
['В каком году', 'началась', 'Варшавская заутреня', '?']
['В каком году', 'началась', 'Битва при Нанси', '?']
['В каком году', 'начались', 'Бои за Комсомольское', '?']
['В каком году', 'началась', 'Битва под Монте-Кассино', '?']
['В каком году', 'началась', 'Курская битва', '?']
['В каком году', 'началось', 'Сражение в заливе Лейте', '?']
['В каком году', 'началась', 'Битва при Прейсиш-Эйлау', '?']
['В каком году', 'началась', 'Битва при Раве-Русской', '?']
['В каком году', 'началась', 'Ржевская битва', '?']
['В каком году', 'началось', 'Сражение при Четати', '?']
['В каком году', 'началась', 'Битва у Корсакова', '?']
['В каком году', 'началось', 'Красногорское сражение', '?']
['В каком году', 'началось', 'Российско-грузинский бой в Чёрном море', '?']
['В к

In [36]:
# # save results

# import os

# # os.chdir("../")

# os.chdir("questions_open")

# with open('start_bitva8--.json', "w", encoding="utf-8") as f:
#     json.dump(questions_15, f, ensure_ascii=False, indent=4)

In [108]:
datefreq = {}
datewhat = {}
for rel, what in dates:
    if rel not in datefreq:
        datefreq[rel] = 0 
    datefreq[rel] += 1
    
    
    if rel not in datewhat:
        datewhat[rel] = []
    datewhat[rel].append(what)

In [109]:
datefreq['дата принятия в эксплуатацию (P729)']

62

In [110]:
# sort by frequency

{k: v for k, v in sorted(datefreq.items(), key=lambda item: item[1], reverse=True)}

{'дата рождения (P569)': 15219,
 'дата смерти (P570)': 14046,
 'дата основания/создания/возникновения (P571)': 2468,
 'дата начала (P580)': 914,
 'дата окончания (P582)': 848,
 'дата прекращения существования (P576)': 688,
 'дата публикации (P577)': 462,
 'дата официального открытия (P1619)': 116,
 'дата принятия в эксплуатацию (P729)': 62,
 'дата вывода из эксплуатации (P730)': 14,
 'дата погребения или кремации (P4602)': 12,
 'дата запуска космического корабля (P619)': 9,
 'дата открытия (P575)': 8,
 'дата крещения (P1636)': 8,
 'дата посадки (P620)': 8,
 'дата исчезновения (P746)': 4,
 'минимальный возраст кандидата (P2998)': 4,
 'дата официального закрытия (P3999)': 3,
 'дата схода с орбиты (P621)': 1,
 'дата коммерциализации (P5204)': 1}

In [111]:
datewhat['дата прекращения существования (P576)']

['дивизия',
 'милиция',
 'комитет',
 'воинское звание',
 'китайская династия',
 'историческое государство',
 'ханство',
 'аэродром',
 'воинское формирование',
 'дивизия',
 'княжество',
 'историческое государство',
 'суверенное государство',
 'воинское формирование',
 'историческое государство',
 'орган публичной власти',
 'сухопутные войска',
 'центральный банк',
 'историческое государство',
 'суверенное государство',
 'историческое государство',
 'губерния Российской империи',
 'историческое государство',
 'суверенное государство',
 'полевая армия',
 'государство',
 'академическое сообщество',
 'пехотная дивизия',
 'федеративная монархия',
 'исторические народы',
 'журнал',
 'Полковое устройство Украины',
 'историческое государство',
 'государство',
 'губерния',
 'историческое государство',
 'центральный комитет',
 'политическая партия',
 'династия',
 'международная организация',
 'город',
 'фронт',
 'военный округ',
 'союзная республика в составе СССР',
 'военно-учебное заведение',
 

In [93]:
# import collections

# dates_sorted = collections.OrderedDict(sorted(datefreq.items()))

# dates_sorted


In [66]:
#unique data + what
dates_wh = list(set(dates))

In [68]:
len(dates_wh)

1264

In [67]:
dates_wh

[('дата окончания (P582)', 'Ледниковый период'),
 ('дата основания/создания/возникновения (P571)', 'орден'),
 ('дата окончания (P582)', 'Крестовые походы'),
 ('дата основания/создания/возникновения (P571)', 'область'),
 ('дата окончания (P582)', 'военный приказ'),
 ('дата прекращения существования (P576)', 'оккупированные территории'),
 ('дата основания/создания/возникновения (P571)',
  'роспуск административно-территориального образования'),
 ('дата основания/создания/возникновения (P571)', 'метрополитен'),
 ('дата основания/создания/возникновения (P571)', 'центральный банк'),
 ('дата официального открытия (P1619)', 'вантовый мост'),
 ('дата официального открытия (P1619)', 'храм'),
 ('дата начала (P580)', 'экспедиция'),
 ('дата публикации (P577)', 'театральная пьеса'),
 ('дата прекращения существования (P576)', 'колония'),
 ('дата начала (P580)', 'научная командировка'),
 ('дата основания/создания/возникновения (P571)', 'деревня'),
 ('дата основания/создания/возникновения (P571)', 'Сх

## Дата, достать

In [65]:
# collect relation-what pairs

def make_rel_what(big_list, rel):
    category_of = []
    for di in big_list:
        for key, value in di['wikidata'].items():
    #         print(key)
            if 'what' not in list(di.keys()) or di['what'] == None:
                pass
            else:
                if key == rel:
                    category_of.append((key,di['what']))
    #             print(di['what'])
    return category_of


# sort tuples by frequency

def sort_fr(category_of):
        
    rel_freq = {}
    rel_what = {}
    for rel_what in category_of:
        if rel_what not in rel_freq:
            rel_freq[rel_what] = 0 
        rel_freq[rel_what] += 1
    return {k: v for k, v in sorted(rel_freq.items(), key=lambda item: item[1], reverse=True)}

In [66]:
rel = 'момент времени (P585)'

sort_fr(make_rel_what(big_list, rel))

{('момент времени (P585)', 'битва'): 311,
 ('момент времени (P585)', 'год'): 66,
 ('момент времени (P585)', 'мирный договор'): 47,
 ('момент времени (P585)', 'хроника в Википедии'): 29,
 ('момент времени (P585)', 'международный договор'): 29,
 ('момент времени (P585)', 'информационный список'): 27,
 ('момент времени (P585)', 'статья-список в проекте Викимедиа'): 25,
 ('момент времени (P585)', 'конфликт'): 18,
 ('момент времени (P585)', 'осада'): 17,
 ('момент времени (P585)', 'война'): 16,
 ('момент времени (P585)', 'десятилетие'): 15,
 ('момент времени (P585)', 'революция'): 13,
 ('момент времени (P585)', 'восстание'): 11,
 ('момент времени (P585)', 'событие'): 10,
 ('момент времени (P585)', 'век'): 8,
 ('момент времени (P585)', 'морское сражение'): 8,
 ('момент времени (P585)', 'бойня'): 8,
 ('момент времени (P585)', 'розыгрыш спортивного турнира'): 7,
 ('момент времени (P585)', 'государственный переворот'): 6,
 ('момент времени (P585)', 'референдум'): 5,
 ('момент времени (P585)', '

In [107]:
rel = 'момент времени (P585)'
what = 'международный договор'

import re


questions_18 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()) or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был заключён', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'был заключён', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
                    print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_18 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_18)


['В каком году', 'был подписан', 'Бартенштейнская конвенция', '?'] ['Бартенштейнская конвенция', 'был подписан', 'в', '1807', 'году', '.']
['В каком году', 'был подписан', 'Батумский договор', '?'] ['Батумский договор', 'был подписан', 'в', '1918', 'году', '.']
['В каком году', 'был подписан', 'Франко-русский союз', '?'] ['Франко-русский союз', 'был подписан', 'в', '1894', 'году', '.']
['В каком году', 'был подписан', 'Георгиевский трактат', '?'] ['Георгиевский трактат', 'был подписан', 'в', '1783', 'году', '.']
['В каком году', 'был подписан', 'Шёнбруннский договор', '?'] ['Шёнбруннский договор', 'был подписан', 'в', '1805', 'году', '.']
['В каком году', 'был подписан', 'Продажа Аляски', '?'] ['Продажа Аляски', 'был подписан', 'в', '1867', 'году', '.']
['В каком году', 'был подписан', 'Устав ООН', '?'] ['Устав ООН', 'был подписан', 'в', '1945', 'году', '.']
['В каком году', 'был подписан', 'Аккерманская конвенция', '?'] ['Аккерманская конвенция', 'был подписан', 'в', '1826', 'году', '

20

In [109]:
# get unique!!! best needed

import ast
questions_18 = [ast.literal_eval(el1) for el1 in set([str(el2) for el2 in questions_18])]
# un

In [103]:
# questions_18

In [108]:
# beautify open questions
def boc(list_of_dicts):
    for d in list_of_dicts:
        # make a list of the entity string
        words = d['open_q'][-2].split()
        if number(words[0]) == 'plur':
            d['open_q'][-3] = d['closed_q'][-5] = "были заключены"
        elif words[0].endswith('а') or words[0].endswith('я') or (gender(words[-1]) == 'femn' and case(words[-1]) == 'nomn'):
            d['open_q'][-3] = d['closed_q'][-5] = "была заключена"
        elif any([f for f in words if f.endswith('е')]):
            d['open_q'][-3] = d['closed_q'][-5] = "было заключено"
            
        else:
            d['open_q'][-3] = d['closed_q'][-5] = "был заключён"
        print(d['open_q'])

In [110]:
boc(questions_18)

['В каком году', 'был заключён', 'Договор между Российской Федерацией и Республикой Крым о принятии в Российскую Федерацию Республики Крым', '?']
['В каком году', 'был заключён', 'Договор о ликвидации ракет средней и меньшей дальности', '?']
['В каком году', 'был заключён', 'Георгиевский трактат', '?']
['В каком году', 'была заключена', 'Бартенштейнская конвенция', '?']
['В каком году', 'было заключено', 'Соглашение СССР и Финляндии об Аландских островах', '?']
['В каком году', 'были заключены', 'Локарнские договоры', '?']
['В каком году', 'было заключено', 'Договор о запрещении испытаний ядерного оружия в атмосфере, космическом пространстве и под водой', '?']
['В каком году', 'был заключён', 'Устав ООН', '?']
['В каком году', 'был заключён', 'Батумский договор', '?']
['В каком году', 'был заключён', 'Договор об ограничении систем противоракетной обороны', '?']
['В каком году', 'была заключена', 'Конвенция о защите прав человека и основных свобод', '?']
['В каком году', 'была заключена

In [111]:
# # save results

# import os

# # os.chdir("../")

# os.chdir("questions_open")

with open('moment_treaty20.json', "w", encoding="utf-8") as f:
    json.dump(questions_18, f, ensure_ascii=False, indent=4)

In [ ]:
rel = 'дата основания/создания/возникновения (P571)'
what = 'храм'

import re


questions_18 = []

def make_q_p(big_list, rel, what, t):
#     print(what)
    questions = []
    
    for i, di in enumerate(big_list):
        
    #     for key, value in di['wikidata'].items():
        if 'what' not in list(di.keys()):# or '(' in di['ent']:
            pass
        else:
            
            if di['what'] == what and rel in di['wikidata'] and type(di['wikidata'][rel]) != list:
#                 print(di)
                try:
#                     if type(di['wikidata']['жанр (P136)']) ==list:
#                         zhanr = di['wikidata']['жанр (P136)'][0].split('(')[0].strip()
#                     else:
#                         zhanr = di['wikidata']['жанр (P136)'].split('(')[0].strip()
                    r = re.search(r"\((.+)\)", rel)
                    r = r.group(1)
#                     print('r',r, di['wikidata'][rel]) 
#                     print(re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2))
                    
#                     a = re.search(r"\((.+)\)", di['wikidata'][rel])
#                     a = r.group(1) #'answer_code':a, \
#                     answer_open = di['wikidata'][rel].split('(')[0].strip()
                    answer_open = re.search(r'(\+)(.*?)—? ?(-)', di['wikidata'][rel]).group(2)
#                     ['создатель (P170)', di['wikidata']['создатель (P170)']]
#                     atr = di['wikidata'][rel].split('(')[1].strip()[:-1]
#                     print(' answer_open atr',answer_open)
                    d = {'type':t, \
                        'what':di['what'], \
                        'ent':di['ent'], \
                        'Q':di['wikibase'], \
                        'rel_r':rel, \
                        'rel': r, \
                        'atr': 'date', \
                        'open_q':['В каком году', 'был основан', di['ent'],'?'], \
                        
                        'answer_open': answer_open, \
                        'closed_q':[di['ent'], 'был основан', 'в',  answer_open, 'году', '.'], \
                        'answer_closed':'Да'
                                       }
                    questions.append(d)
#                     print(di)
#                     print(di['wikidata'])
#                     print(d['open_q'], d['closed_q'])
                except:
                    pass
        if len(questions) == 1000:
            break
    return questions

questions_18 = make_q_p(big_list, rel, what, 'date')
# questions_all += questions_2
len(questions_18)


In [ ]:
relations = ["дата начала (P580)","дата окончания (P582)", "дата основания/создания/возникновения (P571)", "дата прекращения существования (P576)","дата рождения (P569)","дата смерти (P570)", ]

In [ ]:
import wptools

In [1]:
# #look

# wiki_ent = wptools.page("Вильсон, Вудро", lang='ru').get_wikidata()
# wiki_ent.data['wikidata']#['часть от (P361)']

In [11]:
# wiki_ent.data['aliases']

In [9]:
# wiki_ent.data['claims']

In [6]:
wiki_ent.data['description']

'28-й президент США'

In [7]:
wiki_ent.data['label']

'Вудро Вильсон'

In [8]:
wiki_ent.data['what']

'человек'

In [85]:
def get_info(ent):
    try:
        wiki_ent = wptools.page(ent, lang='ru').get_wikidata()
    except(LookupError, ValueError):
        return {'LookupError':ent}
    try:
        wiki_ent = wptools.page(ent, lang='ru').get_wikidata()
    except(ValueError):
        print(ValueError)
        return {'ValueError':ent}
    infotype = ['wikibase', 'what','wikidata', 'aliases', 'claims', 'description', 'label']
    
    info_d = {}
    info_d.update({'ent':ent})
    
    for t in infotype:
        try:
            inf = wiki_ent.data[t]
            info_d.update({t:inf})
        except(KeyError):
            continue
    return info_d

In [86]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

# for x in batch(range(0, 10), 3):
#     print(x)

In [90]:
import json

h = json.load(open("man_ent_list_30.json", "r", encoding="utf-8"))

In [104]:
part1 = h[24650:]

In [106]:
#  collect keys and wikidata and other wiki data
# заводить частями и сохранять частями

from tqdm import tnrange, tqdm_notebook # might work
from time import sleep
import os
from datetime import datetime

count = 0


dirName = 'Processed_from20000'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")
    

for j in tqdm_notebook(batch(part1,30), desc='a loop'):
    info = []
    errors = []
    for i in j:
        info.append(get_info(i))
        
    name = datetime.utcnow().strftime('%Y-%m-%d %H_%M_%S.%f')[:-3]
    filename = "Processed_from20000/%s.json"%name
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=4)
    sleep(0.01)

In [41]:
len(info)

20

In [70]:
info

[{'ent': 'Приволжский военный округ',
  'wikibase': 'Q4377891',
  'what': 'военный округ',
  'wikidata': {'государство (P17)': 'Россия (Q159)',
   'изображение (P18)': 'Приволжский военный округ.jpg',
   'часть от (P361)': 'Министерство обороны Российской Федерации (Q1146492)',
   'это частный случай понятия (P31)': 'военный округ (Q580112)',
   'полученные награды (P166)': 'орден Красного Знамени (Q337463)',
   'дата основания/создания/возникновения (P571)': '+1918-05-04T00:00:00Z',
   'дата прекращения существования (P576)': '+2001-11-30T00:00:00Z'},
  'aliases': ['ПриВО'],
  'claims': {'P17': ['Q159'],
   'P18': ['Приволжский военный округ.jpg'],
   'P361': ['Q1146492'],
   'P31': ['Q580112'],
   'P166': ['Q337463'],
   'P571': ['+1918-05-04T00:00:00Z'],
   'P576': ['+2001-11-30T00:00:00Z']},
  'description': None,
  'label': 'Приволжский военный округ'},
 {'ent': 'История Смоленска',
  'wikibase': 'Q20982807',
  'what': 'хронология',
  'wikidata': {'это частный случай понятия (P31)